# Eat Safe, Love

## Notebook Set Up

In [13]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [14]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [15]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [16]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [17]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [18]:
# Find the establishments with a hygiene score of 20
query ={'scores.Hygiene':20}
fields = {'BusinessName': 1, 'scores.Hygiene': 1}

# Use count_documents to display the number of documents in the result
print("Number of establishments having 20 score for Hygiene:",  establishments.count_documents(query))
# Display the first document in the results using pprint
results=establishments.find(query,fields)
pprint(results[0])

Number of establishments having 20 score for Hygiene: 41
{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('667a118209928dae2f0c2671'),
 'scores': {'Hygiene': 20}}


In [19]:
# Convert the result to a Pandas DataFrame
Hygiene_df=pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Number of records in the DataFrame:",len(Hygiene_df))
# Display the first 10 rows of the DataFrame
Hygiene_df.head(10)

Number of records in the DataFrame: 41


_id               BusinessName           scores
0  667a118209928dae2f0c2671        The Chase Rest Home  {'Hygiene': 20}
1  667a118209928dae2f0c29ef                 Brenalwood  {'Hygiene': 20}
2  667a118209928dae2f0c2cf9              Melrose Hotel  {'Hygiene': 20}
3  667a118209928dae2f0c2ee5              Seaford Pizza  {'Hygiene': 20}
4  667a118209928dae2f0c2ef6              Golden Palace  {'Hygiene': 20}
5  667a118309928dae2f0c3894           Ashby's Butchers  {'Hygiene': 20}
6  667a118309928dae2f0c3ab5  South Sea Express Cuisine  {'Hygiene': 20}
7  667a118309928dae2f0c4fe0              Golden Palace  {'Hygiene': 20}
8  667a118309928dae2f0c5427             The Tulip Tree  {'Hygiene': 20}
9  667a118309928dae2f0c5c45                      F & S  {'Hygiene': 20}

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [20]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query ={'LocalAuthorityName': {'$regex':'London'},'RatingValue': {'$gte': 4}}
fields = {'BusinessName': 1, 'RatingValue': 1,'LocalAuthorityName':1}

# Use count_documents to display the number of documents in the result
print("Number of establishments in Londn having rating value 5 or more:",establishments.count_documents(query))
# Display the first document in the results using pprint
results=establishments.find(query,fields)
pprint(results[0])

Number of establishments in Londn having rating value 5 or more: 33
{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': 4,
 '_id': ObjectId('667a118309928dae2f0c4088')}


In [9]:
# Convert the result to a Pandas DataFrame
london_df=pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Number of records in the DataFrame:",len(london_df))
# Display the first 10 rows of the DataFrame
london_df.head(10)

Number of records in the DataFrame: 33


_id                          BusinessName  \
0  6676d910b736bafe31d9e9fa                             Charlie's   
1  6676d910b736bafe31d9ed23               Mv City Cruises Erasmus   
2  6676d910b736bafe31d9f871             Benfleet Motor Yacht Club   
3  6676d910b736bafe31da066f              Tilbury Seafarers Centre   
4  6676d910b736bafe31da0671  Coombs Catering t/a The Lock and Key   
5  6676d910b736bafe31da0d79                            Mv Valulla   
6  6676d910b736bafe31da2e88                         Tereza Joanne   
7  6676d910b736bafe31da324c                      Brick Lane Brews   
8  6676d910b736bafe31da3266                              WH Smith   
9  6676d910b736bafe31da3268         The Nuance Group (UK) Limited   

   RatingValue          LocalAuthorityName  
0            4  City of London Corporation  
1            5  City of London Corporation  
2            4  City of London Corporation  
3            5  City of London Corporation  
4            5  City of London Corporation  
5            5  City of London Corporation  
6            5  City of London Corporation  
7            4  City of London Corporation  
8            5  City of London Corporation  
9            5  City of London Corporation

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [22]:
# find the geo locations of the "Penang Flavours"
location=establishments.find_one({'BusinessName':'Penang Flavours'},{'geocode.latitude':1,'geocode.longitude':1})

# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score


degree_search = 0.01
latitude =location['geocode']['latitude']
longitude =location['geocode']['longitude']

query ={'RatingValue': 5,
        'geocode.latitude':{'$gte': latitude - degree_search, '$lte': latitude + degree_search},
        'geocode.longitude':{'$gte': longitude - degree_search, '$lte': longitude + degree_search}
       }
sort = [('scores.Hygiene', -1)]
limit =5

# Print the results
pprint(list(establishments.find(query).sort(sort).limit(limit)))

[{'AddressLine1': '101 Plumstead High Street',
  'AddressLine2': '',
  'AddressLine3': 'Plumstead',
  'AddressLine4': 'Greenwich',
  'BusinessName': 'Lucky Food & Wine',
  'BusinessType': 'Retailers - other',
  'BusinessTypeID': 4613,
  'ChangesByServerID': 0,
  'Distance': 4647.024793263386,
  'FHRSID': 695287,
  'LocalAuthorityBusinessID': 'PI/000182135',
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': False,
  'Phone': '',
  'PostCode': 'SE18 1SB',
  'RatingDate': '2022-06-25T00:00:00',
  'RatingKey': 'fhrs_5_en-gb',
  'RatingValue': 5,
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('667a118309928dae2f0c7f6f'),
  'geocode': {'latitude': 51.4878934, 'longitude': 0.0910104},
  'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/695287',
             'rel': 'self'}],
  'meta': {'dataSource

In [23]:
# Convert result to Pandas DataFrame
results=establishments.find(query).sort(sort).limit(limit)
nearest_df=pd.DataFrame(results)
nearest_df

_id   FHRSID  ChangesByServerID  \
0  667a118309928dae2f0c7f6f   695287                  0   
1  667a118309928dae2f0c7f59   694606                  0   
2  667a118309928dae2f0c7f91   695033                  0   
3  667a118309928dae2f0c7f8c   695566                  0   
4  667a118309928dae2f0c7e7b  1069652                  0   

  LocalAuthorityBusinessID                        BusinessName  \
0             PI/000182135                   Lucky Food & Wine   
1             PI/000116584                Fineway Cash & Carry   
2             PI/000172986                  Everest Stores Ltd   
3             PI/000188497                     Premier Express   
4             PI/000206841  TIWA N TIWA African Restaurant Ltd   

              BusinessType  BusinessTypeID               AddressLine1  \
0        Retailers - other            4613  101 Plumstead High Street   
1        Retailers - other            4613  112 Plumstead High Street   
2        Retailers - other            4613  104 Plumstead High Street   
3        Retailers - other            4613  102 Plumstead High Street   
4  Restaurant/Cafe/Canteen               1  152 Plumstead High Street   

  AddressLine2 AddressLine3  ...             LocalAuthorityWebSite  \
0                 Plumstead  ...  http://www.royalgreenwich.gov.uk   
1                 Plumstead  ...  http://www.royalgreenwich.gov.uk   
2                 Plumstead  ...  http://www.royalgreenwich.gov.uk   
3                 Plumstead  ...  http://www.royalgreenwich.gov.uk   
4                 Plumstead  ...  http://www.royalgreenwich.gov.uk   

     LocalAuthorityEmailAddress  \
0  health@royalgreenwich.gov.uk   
1  health@royalgreenwich.gov.uk   
2  health@royalgreenwich.gov.uk   
3  health@royalgreenwich.gov.uk   
4  health@royalgreenwich.gov.uk   

                                              scores  SchemeType  \
0  {'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...        FHRS   
1  {'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...        FHRS   
2  {'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...        FHRS   
3  {'Hygiene': 5, 'Structural': 0, 'ConfidenceInM...        FHRS   
4  {'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...        FHRS   

                                            geocode RightToReply     Distance  \
0  {'longitude': 0.0910104, 'latitude': 51.4878934}               4647.024793   
1  {'longitude': 0.0909382, 'latitude': 51.4876089}               4647.016995   
2  {'longitude': 0.0903232, 'latitude': 51.4876718}               4647.041728   
3  {'longitude': 0.0904552, 'latitude': 51.4877234}               4647.038807   
4  {'longitude': 0.0927429, 'latitude': 51.4870351}               4646.930146   

  NewRatingPending                                               meta  \
0            False  {'dataSource': None, 'extractDate': '0001-01-0...   
1            False  {'dataSource': None, 'extractDate': '0001-01-0...   
2            False  {'dataSource': None, 'extractDate': '0001-01-0...   
3            False  {'dataSource': None, 'extractDate': '0001-01-0...   
4            False  {'dataSource': None, 'extractDate': '0001-01-0...   

                                               links  
0  [{'rel': 'self', 'href': 'http://api.ratings.f...  
1  [{'rel': 'self', 'href': 'http://api.ratings.f...  
2  [{'rel': 'self', 'href': 'http://api.ratings.f...  
3  [{'rel': 'self', 'href': 'http://api.ratings.f...  
4  [{'rel': 'self', 'href': 'http://api.ratings.f...  

[5 rows x 28 columns]

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [24]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match': {'scores.Hygiene':0}}

# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id': "$LocalAuthorityName", 'count': { '$sum': 1 }}}

# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': { 'count': -1 }}

# Print the number of documents in the result
pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))
print("Number of records :", len(results))

# Print the first 10 results
results[0:10]

Number of records : 55


[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]

In [25]:
# Convert the result to a Pandas DataFrame
count_df=pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Number of records in the DataFrame :", len(count_df))
# Display the first 10 rows of the DataFrame
count_df.head(10)

Number of records in the DataFrame : 55


_id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542